In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline

#Creating a function to load the text and labels from train and test set

In [42]:
import pandas as pd

def get_labels_and_texts(file):
    df = pd.read_csv(file)
    labels = df['sentiment'].values
    texts = df['reviews'].values
    return labels, texts

train_labels, train_texts = get_labels_and_texts('train.csv')
test_labels, test_texts = get_labels_and_texts('test.csv')


In [43]:
train_labels[0:4]

array([1, 0, 1, 0], dtype=int64)

In [44]:
train_texts[0:4]

array(['worked advertised',
       'wanted white dot white bathroom black shown knew realize cord plug black wanted white cord plug',
       'extremely useful simple thing like spotify audible thing like doesnt answer ask nice use',
       'brought replace moved doesnt range previous non refurbished unit seldom able find wi-fi couldnt connect got tired hearing message 34trouble connectingcheck alexa app34 unplugged went troubleshooting reconnected time strong'],
      dtype=object)

In [45]:
# train_labels=train_labels[0:500]

In [46]:
# train_texts=train_texts[0:500]

#text pre-processing

In [47]:
import re

NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')

def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        if isinstance(text, str):  # Check if text is a string
            lower = text.lower()
            no_punctuation = NON_ALPHANUM.sub(r' ', lower)
            no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
            normalized_texts.append(no_non_ascii)
        else:
            # Handle non-string texts here
            # convert to strings:
            normalized_texts.append(str(text))
    return normalized_texts

train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)


In [48]:
train_texts[0]

'worked advertised'

#countvectorizer

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [50]:
X_test

<945x3154 sparse matrix of type '<class 'numpy.int64'>'
	with 9073 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(
    X, train_labels, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.9130434782608695
Accuracy for C=0.05: 0.9148550724637681
Accuracy for C=0.25: 0.927536231884058
Accuracy for C=0.5: 0.9293478260869565
Accuracy for C=1: 0.9347826086956522


In [52]:
lr.predict(X_test[29])

array([1], dtype=int64)

In [53]:
test_labels[29]

1

In [54]:
test_texts[29]

'work amazon netflix let viewing platform like vudu shut hulu played couple minute yes directly plugged wall low power source issue checked outlet voltage issue typically answer issueover quality picture quiet user friendly platform use fix issue tv house sits bad relay like amazon product having alexa nice'